# **Start**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Uninstall existing scikit-learn to avoid conflicts
!pip uninstall -y scikit-learn
# Install specific versions of libraries to avoid conflicts
!pip install scikit-learn==1.5.2
!pip install bayesian-optimization
!pip install optuna
!pip install gpboost
!pip install shap
!pip install ngboost
!pip install dask[dataframe]
!pip install torch seaborn
!pip install lightgbm
!pip install xgboost
!pip install lime
!pip install interpret
!pip install optunahub
!pip install cmaes
!pip install plotly kaleido
!pip install openpyxl
!pip install -U kaleido
!pip install properscoring
!pip install XlsxWriter
!pip install cython
!pip install pgbm
!pip install torch
!pip install cp
!pip install mapie
!pip install torch skorch puncc
# Reinstall scikit-learn to the version required by ngboost
!pip uninstall -y scikit-learn
!pip install scikit-learn==1.6.1
# Reinstall numpy first
!pip install numpy==1.26.4  # Use the version compatible with catboost
# Reinstall catboost
!pip install catboost

Found existing installation: scikit-learn 1.6.1
Uninstalling scikit-learn-1.6.1:
  Successfully uninstalled scikit-learn-1.6.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 32.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 9.5 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4030 sha256=02584072bf43f75e6185933a1c2a8affc0244a5e2d1a163373102d95b82b0a3b
  Stored in directory: /root/.cache/pip/wheels/8b/67/f4/2caaae2146198dcb824f31a303833b07b14a5ec863fb3acd7b
Su

In [ ]:
# Restart the runtime to apply changes
import os
os._exit(00)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ngboost
from scipy.stats import randint
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.svm import SVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from gpboost import GPBoostRegressor
from ngboost import NGBRegressor
import optuna
import optunahub
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from interpret import show
from interpret.blackbox import LimeTabular, ShapKernel
from optuna.samplers import RandomSampler
import random
import time
from ngboost.distns import Normal
from ngboost.scores import LogScore
from scipy.stats import norm
from interpret import set_visualize_provider
from interpret.provider import InlineProvider
from interpret.glassbox import ExplainableBoostingRegressor
from interpret import show
import plotly.express as px
from io import BytesIO
from openpyxl import Workbook, load_workbook
import os
from openpyxl.drawing.image import Image as openpyxlImage
import warnings
import xlsxwriter
from openpyxl.drawing.image import Image
from pgbm.sklearn import HistGradientBoostingRegressor
import torch
from pgbm.torch import PGBM
import plotly.graph_objects as go
warnings.filterwarnings('ignore')
import pickle
import json

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
Using /root/.cache/torch_extensions/py311_cu124 as PyTorch extensions root...
Creating extension directory /root/.cache/torch_extensions/py311_cu124/split_decision...
Emitting ninja build file /root/.cache/torch_extensions/py311_cu124/split_decision/build.ninja...
Building extension module split_decision...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module split_decision...
Using /root/.cache/torch_extensions/py311_cu124 as PyTorch extensions root...
No modifications detected for re-loaded extension module split_decision, skipping build step...
Loading extension module split_decision...


In [2]:
train_data_path = "./drive/MyDrive/WATER QUALITY/EWQI_uncertainity/train.csv"
test_data_path = "./drive/MyDrive/WATER QUALITY/EWQI_uncertainity/test.csv"
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)
print("Training data loaded successfully.")
print("Test data loaded successfully.")

Training data loaded successfully.
Test data loaded successfully.


In [3]:
print("\nShape of training data:", train_data.shape)
print("First 5 rows of training data:\n", train_data.head(5))
print("\nShape of test data:", test_data.shape)
print("First 5 rows of test data:\n", test_data.head(5))


Shape of training data: (47, 15)
First 5 rows of training data:
      pH   TDS    EC    TH   ca    mg  CO3  HCO3   NA    K  CHLORIDE  SULPHATE  \
0  7.90   552   865   452  272   180  0.2   4.5  260   44       375       681   
1  7.52  1985  2256  1054  529   525  0.0  15.0  272   47       728       324   
2  7.65  1865  2032   636  371   265  0.0   9.0  373   25       713       503   
3  7.25  4855  7856  2050  795  1255  0.0  12.5  335  106        81       676   
4  7.96  2855  3696   865  310   555  0.3   4.6  184  106       698       324   

   NITRATE  FLUORIDE  Pre monsoon EWQI   
0       60      0.40              70.52  
1       10      0.65             138.78  
2       54      0.85             113.07  
3       17      0.69             266.32  
4       25      1.20             158.05  

Shape of test data: (47, 15)
First 5 rows of test data:
      pH   TDS    EC    TH     ca      mg  CO3  HCO3   NA    K  CHLORIDE  \
0  8.38   625   975   320  120.0   200.0  0.0   5.8  280   35 

In [4]:
X_train = train_data.iloc[:, :-1]
y_train = train_data.iloc[:, -1]
X_test = test_data.iloc[:, :-1]
y_test = test_data.iloc[:, -1]
x_test= X_test
print("\nShape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)


Shape of X_train: (47, 14)
Shape of y_train: (47,)
Shape of X_test: (47, 14)
Shape of y_test: (47,)


In [5]:
# Apply z-score normalization
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Print the first five rows of the normalized data
print("\nFirst five rows of normalized X_train:")
print(X_train[:14])

print("\nFirst five rows of normalized X_test:")
print(X_test[:14])


First five rows of normalized X_train:
[[ 0.3797749  -1.15230942 -0.94309903 -0.47923131 -0.09229732 -0.56852482
  -0.03922784 -1.18768739 -0.02713192 -0.87884263 -0.2770724   1.15873613
   1.41276153 -1.25316406]
 [-0.78765707 -0.2953837  -0.52059702  0.45803078  1.31357822  0.10564733
  -0.50015494  1.61405127  0.07159325 -0.77136925  1.29937074 -0.45273785
  -1.61366471 -0.53733141]
 [-0.38827245 -0.36714301 -0.58863473 -0.19275917  0.44926563 -0.40242444
  -0.50015494  0.01305775  0.90253007 -1.55950739  1.23238307  0.35525611
   1.04959038  0.03533472]
 [-1.61714822  1.42085972  1.18034563  2.00871688  2.76868675  1.53215652
  -0.50015494  0.94697063  0.58990037  1.34227396 -1.59003071  1.13616647
  -1.18996504 -0.42279818]
 [ 0.56410627  0.22487128 -0.08321177  0.16377408  0.11557533  0.16427099
   0.19123571 -1.16100417 -0.6523913   1.34227396  1.1653954  -0.45273785
  -0.70573684  1.03750043]
 [ 0.84060332  4.291232    3.81954041  3.87701339  1.42845521  4.35586306
  -0.500154

# **Functions**

In [6]:
# Define the model classes
model_classes = {
    'Random Forest': RandomForestRegressor,
    'Gradient Boosting': GradientBoostingRegressor,
    'XGBoost': XGBRegressor,
    'LightGBM': LGBMRegressor,
    'GPBoost': GPBoostRegressor,
    'CatBoost': CatBoostRegressor,
    'NGBoost': NGBRegressor
}

feature_names = ['pH', 'TDS', 'EC', 'TH', 'ca', 'mg', 'CO3', 'HCO3', 'NA', 'K', 'CHLORIDE', 'SULPHATE', 'NITRATE', 'FLUORIDE']



In [7]:
def plot_best_scores(best_scores_ran, excel_file_path):
    # Extract the best pruner for each model based on RMSE and correlation coefficient
    best_rmse_scores = {}
    best_corr_coef_scores = {}

    for (model_name, pruner_name), scores in best_scores_ran.items():
        # Initialize if not already present
        if model_name not in best_rmse_scores:
            best_rmse_scores[model_name] = (scores['test_rmse'], pruner_name)
        if model_name not in best_corr_coef_scores:
            best_corr_coef_scores[model_name] = (scores['test_corr_coef'], pruner_name)

        # Update if better scores are found
        if scores['test_rmse'] < best_rmse_scores[model_name][0]:
            best_rmse_scores[model_name] = (scores['test_rmse'], pruner_name)
        if scores['test_corr_coef'] > best_corr_coef_scores[model_name][0]:
            best_corr_coef_scores[model_name] = (scores['test_corr_coef'], pruner_name)

    # Prepare data for plotting
    model_names_rmse = [f"{model} ({pruner})" for model, (rmse, pruner) in best_rmse_scores.items()]
    rmse_values = [rmse for rmse, _ in best_rmse_scores.values()]

    model_names_corr = [f"{model} ({pruner})" for model, (corr, pruner) in best_corr_coef_scores.items()]
    corr_values = [corr for corr, _ in best_corr_coef_scores.values()]

    # Plot RMSE
    plt.figure(figsize=(12, 6))
    bars_rmse = plt.bar(model_names_rmse, rmse_values, color='skyblue')

    # Highlight the best model
    best_rmse_index = np.argmin(rmse_values)
    bars_rmse[best_rmse_index].set_color('orange')

    # Annotate the bars with the RMSE scores
    for i, bar in enumerate(bars_rmse):
        plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() - 0.05,
                 f'{rmse_values[i]:.5f}', ha='center', va='bottom', color='black')

    # Add labels and title for the RMSE bar chart
    plt.xlabel('Model (Pruner)')
    plt.ylabel('Test RMSE')
    plt.title('Best Test RMSE for Each Model')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    rmse_image_path = 'rmse_plot.png'
    plt.savefig(rmse_image_path)
    plt.close()

    # Plot Correlation Coefficient
    plt.figure(figsize=(12, 6))
    bars_corr = plt.bar(model_names_corr, corr_values, color='lightgreen')

    # Highlight the best model
    best_corr_index = np.argmax(corr_values)
    bars_corr[best_corr_index].set_color('orange')

    # Annotate the bars with the correlation coefficient scores
    for i, bar in enumerate(bars_corr):
        plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() - 0.05,
                 f'{corr_values[i]:.5f}', ha='center', va='bottom', color='black')

    # Add labels and title for the correlation coefficient bar chart
    plt.xlabel('Model (Pruner)')
    plt.ylabel('Correlation Coefficient')
    plt.title('Best Correlation Coefficient for Each Model')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    corr_image_path = 'corr_plot.png'
    plt.savefig(corr_image_path)
    plt.close()

    # Load the existing Excel file
    workbook = load_workbook(excel_file_path)

    # Create a new sheet for the plots
    sheet_name = 'Best Models Plots'
    if sheet_name in workbook.sheetnames:
        sheet = workbook[sheet_name]
    else:
        sheet = workbook.create_sheet(title=sheet_name)

    # Insert images into the new Excel sheet
    img_rmse = Image(rmse_image_path)
    img_corr = Image(corr_image_path)

    # Insert images
    sheet.add_image(img_rmse, 'A1')
    sheet.add_image(img_corr, 'A20')  # Adjust the position as needed

    # Save the workbook
    workbook.save(excel_file_path)

    # Clean up the image files
    os.remove(rmse_image_path)
    os.remove(corr_image_path)

# Example usage
# plot_best_scores(best_scores_ran, 'path_to_your_excel_file.xlsx')

In [8]:
def generate_interpretml_explanations_summary_pruners(
    results_dict, X_train, y_train, X_test, feature_names, instance_indices=None, excel_file_path=None
):
    if instance_indices is None:
        instance_indices = range(len(X_test))
    elif isinstance(instance_indices, int):
        instance_indices = [instance_indices]

    valid_indices = [idx for idx in instance_indices if 0 <= idx < len(X_test)]
    if not valid_indices:
        print("No valid instance indices provided.")
        return

    if isinstance(X_test, pd.DataFrame):
        instances_to_explain = X_test.iloc[valid_indices]
    else:
        instances_to_explain = X_test[valid_indices]

    best_model_pruners = {}
    for model_key, model_info in results_dict.items():
        if isinstance(model_key, tuple):
            model_name, pruner_name = model_key
        else:
            model_name = model_key
            pruner_name = None

        best_score = model_info.get('best_score')
        if best_score is None:
            print(f"No 'best_score' found for {model_key}. Skipping this combination.")
            continue

        if model_name not in best_model_pruners:
            best_model_pruners[model_name] = {
                'pruner_name': pruner_name,
                'model_info': model_info,
                'best_score': best_score
            }
        else:
            current_best_score = best_model_pruners[model_name]['best_score']
            if best_score < current_best_score:
                best_model_pruners[model_name] = {
                    'pruner_name': pruner_name,
                    'model_info': model_info,
                    'best_score': best_score
                }

    for model_name, info in best_model_pruners.items():
        pruner_name = info['pruner_name']
        model_info = info['model_info']
        best_params = model_info['best_params']
        model_class = model_classes.get(model_name)

        if model_class is None:
            print(f"Model {model_name} is not supported or not available.")
            continue

        if model_name == 'CatBoost':
            best_params['verbose'] = 0

        model = model_class(**best_params)
        model.fit(X_train, y_train)

        def predict_fn(data):
            return model.predict(data)

        if isinstance(X_train, pd.DataFrame):
            data_for_explainer = X_train.values
        else:
            data_for_explainer = X_train

        if isinstance(instances_to_explain, pd.DataFrame):
            data_for_explanation = instances_to_explain.values
        else:
            data_for_explanation = instances_to_explain

        # Generate LIME explanations
        lime_explainer = LimeTabular(
            predict_fn,
            data=data_for_explainer,
            feature_names=feature_names,
            random_state=1,
            mode='regression'
        )
        lime_explanation = lime_explainer.explain_local(data_for_explanation)

        feature_importances_lime = {}
        num_instances = len(valid_indices)

        for idx in range(num_instances):
            explanation = lime_explanation.data(idx)
            for feature_name, feature_score in zip(explanation['names'], explanation['scores']):
                feature_importances_lime[feature_name] = feature_importances_lime.get(feature_name, 0) + abs(feature_score)

        feature_importances_lime = {k: v / num_instances for k, v in feature_importances_lime.items()}
        feature_importances_lime = {k: round(v, 3) for k, v in feature_importances_lime.items()}

        # Generate SHAP explanations using ShapKernel
        try:
            shap_explainer = ShapKernel(predict_fn, data_for_explainer, feature_names=feature_names)
            shap_explanation = shap_explainer.explain_local(data_for_explanation)

            feature_importances_shap = {}
            for idx in range(num_instances):
                explanation = shap_explanation.data(idx)
                for feature_name, feature_score in zip(explanation['names'], explanation['scores']):
                    feature_importances_shap[feature_name] = feature_importances_shap.get(feature_name, 0) + abs(feature_score)

            feature_importances_shap = {k: v / num_instances for k, v in feature_importances_shap.items()}
            feature_importances_shap = {k: round(v, 3) for k, v in feature_importances_shap.items()}
        except Exception as e:
            print(f"Could not compute SHAP values for model {model_name}: {e}")
            continue

        # Plot LIME and SHAP feature importances side by side
        fig, axes = plt.subplots(1, 2, figsize=(34, 36))

        # Plot LIME feature importances
        lime_importances_df = pd.DataFrame.from_dict(
            feature_importances_lime, orient='index', columns=['importance']
        )
        lime_importances_df.sort_values(by='importance', ascending=False, inplace=True)
        lime_importances_df.plot(kind='bar', legend=False, color='skyblue', ax=axes[0])
        axes[0].set_title(f"LIME Feature Importances for {model_name}")
        axes[0].set_ylabel("Average Absolute Importance Score")
        axes[0].set_xlabel("Features")
        axes[0].tick_params(axis='x', rotation=45)

        for p in axes[0].patches:
            height = p.get_height()
            axes[0].annotate(f'{height:.3f}',
                             (p.get_x() + p.get_width() / 2., height),
                             ha='center', va='bottom', fontsize=8)

        # Plot SHAP feature importances
        shap_importances_df = pd.DataFrame.from_dict(
            feature_importances_shap, orient='index', columns=['importance']
        )
        shap_importances_df.sort_values(by='importance', ascending=False, inplace=True)
        shap_importances_df.plot(kind='bar', legend=False, color='orange', ax=axes[1])
        axes[1].set_title(f"SHAP Feature Importances for {model_name}")
        axes[1].set_ylabel("Average Absolute SHAP Value")
        axes[1].set_xlabel("Features")
        axes[1].tick_params(axis='x', rotation=45)

        for p in axes[1].patches:
            height = p.get_height()
            axes[1].annotate(f'{height:.3f}',
                             (p.get_x() + p.get_width() / 2., height),
                             ha='center', va='bottom', fontsize=8)

        plt.tight_layout()

        # Save plots as images
        image_path = f'feature_importances_{model_name}.png'
        fig.savefig(image_path)
        plt.close(fig)

        # Optionally insert images and scores into an Excel file
        if excel_file_path:
            workbook = load_workbook(excel_file_path)
            sheet_name = f'{model_name} Explanations'
            if sheet_name in workbook.sheetnames:
                sheet = workbook[sheet_name]
            else:
                sheet = workbook.create_sheet(title=sheet_name)

            # Insert images into the new Excel sheet
            img = Image(image_path)
            sheet.add_image(img, 'A1')

            # Create a new sheet for feature importance scores
            scores_sheet_name = f'{model_name} Scores'
            if scores_sheet_name in workbook.sheetnames:
                scores_sheet = workbook[scores_sheet_name]
            else:
                scores_sheet = workbook.create_sheet(title=scores_sheet_name)

            # Write LIME scores
            scores_sheet.append(['Feature', 'LIME Importance'])
            for feature, importance in feature_importances_lime.items():
                scores_sheet.append([feature, importance])

            # Write SHAP scores
            scores_sheet.append(['Feature', 'SHAP Importance'])
            for feature, importance in feature_importances_shap.items():
                scores_sheet.append([feature, importance])

            # Save the workbook
            workbook.save(excel_file_path)

            # Clean up the image file
            os.remove(image_path)

# **Autosampler by Optuna**

In [ ]:
# Suppress Optuna's verbose output
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Define objective and metric functions for PGBM
def mseloss_objective(yhat, y, sample_weight=None):
    if not torch.is_tensor(yhat):
        yhat = torch.from_numpy(np.array(yhat)).float()
    if not torch.is_tensor(y):
        y = torch.from_numpy(np.array(y)).float()
    gradient = yhat - y
    hessian = torch.ones_like(yhat)
    return gradient, hessian

def rmseloss_metric(yhat, y, sample_weight=None):
    if not torch.is_tensor(yhat):
        yhat = torch.from_numpy(np.array(yhat)).float()
    if not torch.is_tensor(y):
        y = torch.from_numpy(np.array(y)).float()
    loss = torch.sqrt(torch.mean((yhat - y) ** 2))
    return loss

def hyperparameter_tuning_all(X_train, y_train, X_test, y_test, excel_path):
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)
    y_test = np.array(y_test)

    models = {
        'Random Forest': (RandomForestRegressor, {
            'n_estimators': [100, 200, 300, 500, 700],
            'criterion': ['squared_error', 'absolute_error', 'friedman_mse', 'poisson'],
            'max_depth': [None, 10, 20, 30, 40],
            'min_samples_split': [2, 5, 10, 0.01],
            'min_samples_leaf': [1, 3, 5, 0.01],
            'min_weight_fraction_leaf': [0.0, 0.01, 0.1, 0.2],
            'max_features': [1.0, 'sqrt', 'log2', 0.3, 0.5],
            'max_leaf_nodes': [None, 50, 100, 200],
            'min_impurity_decrease': [0.0, 0.01, 0.1, 0.2],
            'n_jobs': [-1],
            'random_state': [42],
            'verbose': [0],
            'warm_start': [False],
            'ccp_alpha': [0.0, 0.001, 0.01, 0.05, 0.1]
        }),
        'Gradient Boosting': (GradientBoostingRegressor, {
            'loss': ['squared_error', 'absolute_error', 'huber', 'quantile'],
            'learning_rate': [0.01, 0.05, 0.1, 0.2],
            'n_estimators': [100, 200, 300, 500, 700],
            'subsample': [1.0, 0.9, 0.7, 0.5],
            'criterion': ['friedman_mse', 'squared_error'],
            'min_samples_split': [2, 5, 10, 0.01],
            'min_samples_leaf': [1, 3, 5, 0.01],
            'min_weight_fraction_leaf': [0.0, 0.01, 0.05, 0.1],
            'max_depth': [3, 5, 7, 10],
            'min_impurity_decrease': [0.0, 0.01, 0.1],
            'init': [None],
            'random_state': [42],
            'max_features': [None, 'sqrt', 'log2', 0.5],
            'alpha': [0.9, 0.5, 0.1],
            'verbose': [0],
            'max_leaf_nodes': [None, 10, 30, 50],
            'warm_start': [False],
            'validation_fraction': [0.1],
            'n_iter_no_change': [None, 10, 20],
            'tol': [1e-4, 1e-3],
            'ccp_alpha': [0.0, 0.001, 0.01]
        }),
        'XGBoost': (XGBRegressor, {
            'n_estimators': [100, 200, 300, 400, 500],
            'learning_rate': [0.01, 0.05, 0.1, 0.15],
            'max_depth': [3, 5, 7],
            'min_child_weight': [1, 3, 5],
            'gamma': [0, 0.1, 0.5, 1],
            'subsample': [0.5, 0.6, 0.7, 0.8, 0.9],
            'colsample_bytree': [0.5, 0.7, 0.9],
            'colsample_bylevel': [0.5, 0.7, 0.9],
            'reg_alpha': [0, 0.01, 0.1, 1],
            'reg_lambda': [0.1, 1, 5, 10],
            'objective': ['reg:squarederror'],
            'random_state': [42],
            'n_jobs': [-1]
        }),
        'LightGBM': (LGBMRegressor, {
            'n_estimators': [100, 200, 300, 400, 500],
            'learning_rate': [0.01, 0.05, 0.1, 0.15],
            'num_leaves': [15, 31, 63],
            'max_depth': [3, 5, 7, -1],
            'min_child_samples': [1, 5, 10, 20],
            'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
            'colsample_bytree': [0.5, 0.7, 0.9, 1.0],
            'reg_alpha': [0, 0.01, 0.1, 1],
            'reg_lambda': [0, 0.1, 1, 10],
            'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1],
            'bagging_freq': [0, 1, 5],
            'objective': ['regression'],
            'random_state': [42],
            'n_jobs': [-1],
            'verbose': [-1]
        }),
        'GPBoost': (GPBoostRegressor, {
            'n_estimators': [100, 200, 300, 400, 500],
            'learning_rate': [0.01, 0.05, 0.1, 0.15],
            'max_depth': [3, 5, 7, -1],
            'num_leaves': [15, 31, 63],
            'min_child_samples': [1, 5, 10, 20],
            'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
            'colsample_bytree': [0.5, 0.7, 0.9, 1.0],
            'reg_alpha': [0, 0.1, 0.5, 1.0],
            'reg_lambda': [0, 0.1, 0.5, 1.0],
            'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1],
            'random_state': [42],
            'n_jobs': [-1],
            'verbose': [-1]
        }),
        'CatBoost': (CatBoostRegressor, {
            'iterations': [200, 500, 1000],
            'learning_rate': [0.01, 0.03, 0.05, 0.1],
            'depth': [4, 6, 8, 10],
            'l2_leaf_reg': [1, 3, 5, 7, 9],
            'border_count': [32, 64, 128],
            'min_data_in_leaf': [1, 5, 10, 20],
            'rsm': [0.6, 0.8, 1.0],
            'bagging_temperature': [0, 1, 10],
            'random_seed': [42],
            'verbose': [0]
        }),
        'NGBoost': (NGBRegressor, {
            'n_estimators': [200, 500, 1000],
            'learning_rate': [0.01, 0.03, 0.05, 0.1],
            'natural_gradient': [True, False],
            'minibatch_frac': [0.5, 0.7, 0.9, 1.0],
            'col_sample': [0.5, 0.7, 0.9, 1.0],
            'Dist': [Normal],
            'Score': [LogScore],
            'random_state': [42],
            'verbose': [0]
        }),
        'HistGradientBoosting': (HistGradientBoostingRegressor, {
            'learning_rate': [0.01, 0.05, 0.1, 0.15],
            'max_iter': [100, 200, 300, 400, 500],
            'max_depth': [3, 5, 7, None],
            'min_samples_leaf': [5, 10, 20],
            'max_leaf_nodes': [15, 31, 63, None],
            'l2_regularization': [0.0, 0.1, 0.5, 1.0],
            'max_bins': [64, 128, 255],
            'early_stopping': [True, False],
            'validation_fraction': [0.1, 0.2],
            'n_iter_no_change': [5, 10, 15],
            'loss': ['squared_error'],
            'random_state': [42],
            'verbose': [0]
        }),
        'PGBM': (PGBM, {}) # PGBM is handled separately
    }


    pruners = [
        optuna.pruners.MedianPruner(),
        optuna.pruners.NopPruner(),
        optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=3),
        optuna.pruners.PercentilePruner(25.0),
        optuna.pruners.SuccessiveHalvingPruner(),
        optuna.pruners.HyperbandPruner(),
        optuna.pruners.ThresholdPruner(lower=0.1),
        optuna.pruners.WilcoxonPruner()
    ]

    best_scores = {}
    predictions_df = pd.DataFrame({'Actual': y_test})

    for model_name, (model_class, param_space) in models.items():
        for pruner in pruners:
            pruner_name = pruner.__class__.__name__
            print(f"Running Optuna for {model_name} with {pruner_name}...")
            try:
                sampler = optunahub.load_module("samplers/auto_sampler").AutoSampler()
                study = optuna.create_study(direction='minimize', sampler=sampler, pruner=pruner)

                if model_name == 'PGBM':
                    def pgbm_objective(trial):
                        params = {
                            'n_estimators': trial.suggest_categorical('n_estimators', [100, 200, 300, 500]),
                            'learning_rate': trial.suggest_categorical('learning_rate', [0.01, 0.05, 0.1, 0.15]),
                            'max_leaves': trial.suggest_int('max_leaves', 15, 63),
                            'min_split_gain': trial.suggest_categorical('min_split_gain', [0.0, 0.1, 0.5, 1.0]),
                            'reg_lambda': trial.suggest_categorical('reg_lambda', [0.1, 1.0, 5.0, 10.0]),
                            'feature_fraction': trial.suggest_categorical('feature_fraction', [0.5, 0.7, 0.9, 1.0]),
                            'bagging_fraction': trial.suggest_categorical('bagging_fraction', [0.5, 0.7, 0.9, 1.0]),
                            'tree_correlation': trial.suggest_categorical('tree_correlation', [0.0, 0.1, 0.2, 0.3]),
                            'min_data_in_leaf': trial.suggest_categorical('min_data_in_leaf', [3, 5, 10, 20]),
                            'max_bin': trial.suggest_categorical('max_bin', [64, 128, 256]),
                            'distribution': trial.suggest_categorical('distribution', ['normal', 'studentt', 'laplace']),
                            'objective': 'mse',
                            'metric': 'rmse',
                            'random_state': 42,
                            'verbose': 0
                        }

                        model = PGBM()
                        model.train((X_train, y_train), objective=mseloss_objective, metric=rmseloss_metric, params=params)
                        y_pred = model.predict(X_test)
                        mse = mean_squared_error(y_test, y_pred)
                        return mse

                    study.optimize(pgbm_objective, n_trials=50)
                else:
                    def objective(trial, model_name, model_class, param_space):
                        params = {}
                        for key, values in param_space.items():
                            if isinstance(values, list):
                                params[key] = trial.suggest_categorical(key, values)
                            elif isinstance(values, tuple):
                                if len(values) == 2:
                                    params[key] = trial.suggest_float(key, values[0], values[1])
                                elif len(values) == 3 and isinstance(values[2], bool) and values[2]:
                                    params[key] = trial.suggest_int(key, values[0], values[1])
                                else:
                                    raise ValueError(f"Invalid parameter range for {key}")

                        model = model_class(**params)
                        model.fit(X_train, y_train)
                        y_pred = model.predict(X_test)
                        mse = mean_squared_error(y_test, y_pred)
                        return mse

                    study.optimize(lambda trial: objective(trial, model_name, model_class, param_space), n_trials=50)

                best_params = study.best_params
                best_model = model_class(**best_params) if model_name != 'PGBM' else PGBM()
                if model_name == 'PGBM':
                    best_model.train((X_train, y_train), objective=mseloss_objective, metric=rmseloss_metric, params=best_params)
                else:
                    best_model.fit(X_train, y_train)
                y_pred = best_model.predict(X_test)

                mse = mean_squared_error(y_test, y_pred)
                rmse = np.sqrt(mse)
                corr_coef = np.corrcoef(y_test, y_pred)[0, 1]

                # Save predictions to DataFrame
                predictions_df[f'{model_name}_{pruner_name}_Predicted'] = y_pred

                best_scores[(model_name, pruner_name)] = {
                    'best_score': mse,
                    'best_params': best_params,
                    'test_mse': mse,
                    'test_rmse': rmse,
                    'test_corr_coef': corr_coef,
                    'pruner': pruner_name
                }
                print(f"Best MSE for {model_name} with {pruner_name}: {mse} with params: {best_params}")
                print(f"Best RMSE for {model_name} with {pruner_name}: {rmse}")
                print(f"Correlation Coefficient for {model_name} with {pruner_name}: {corr_coef}")
            except Exception as e:
                print(f"Failed to run Optuna for {model_name} with {pruner_name}. Error: {e}")

    # Write predictions to Excel
    with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a') as writer:
        predictions_df.to_excel(writer, sheet_name='Predictions', index=False)

    if best_scores:
        best_model_name, best_pruner_name = min(best_scores, key=lambda k: best_scores[k]['test_mse'])
        best_model_info = best_scores[(best_model_name, best_pruner_name)]
        print(f"\nBest model on test data: {best_model_name} with {best_pruner_name}")
        print(f"Test MSE: {best_model_info['test_mse']}")
        print(f"Test RMSE: {best_model_info['test_rmse']}")
        print(f"Correlation Coefficient: {best_model_info['test_corr_coef']}")
        print(f"Best Parameters: {best_model_info['best_params']}")
        print(f"Pruner Used: {best_model_info['pruner']}")
    else:
        print("No valid model configurations found.")

    return best_scores

# Example usage
best_scores_autosampler = hyperparameter_tuning_all(X_train, y_train, X_test, y_test, "./drive/MyDrive/WATER QUALITY/EWQI_uncertainity/EWQI_ml/HyperParameter_Tuning/test.xlsx")

In [10]:
best_scores_autosampler

{('Random Forest', 'MedianPruner'): {'best_score': 496.75467555727295,
  'best_params': {'n_estimators': 500,
   'criterion': 'squared_error',
   'max_depth': 30,
   'min_samples_split': 2,
   'min_samples_leaf': 1,
   'min_weight_fraction_leaf': 0.01,
   'max_features': 0.5,
   'max_leaf_nodes': 200,
   'min_impurity_decrease': 0.0,
   'n_jobs': -1,
   'random_state': 42,
   'verbose': 0,
   'warm_start': False,
   'ccp_alpha': 0.01},
  'test_mse': 496.75467555727295,
  'test_rmse': 22.28799397786335,
  'test_corr_coef': 0.9568118135387962,
  'pruner': 'MedianPruner'},
 ('Random Forest', 'NopPruner'): {'best_score': 503.0808858431874,
  'best_params': {'n_estimators': 500,
   'criterion': 'friedman_mse',
   'max_depth': 20,
   'min_samples_split': 2,
   'min_samples_leaf': 0.01,
   'min_weight_fraction_leaf': 0.0,
   'max_features': 0.5,
   'max_leaf_nodes': 50,
   'min_impurity_decrease': 0.2,
   'n_jobs': -1,
   'random_state': 42,
   'verbose': 0,
   'warm_start': False,
   'ccp_al

# **Best Model Analysis**

In [20]:
def get_best_models_and_predict(best_scores_autosampler, X_train, y_train, X_test, y_test, file_path):
    # Convert input data to NumPy arrays
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)
    y_test = np.array(y_test)

    # Mapping for model creation based on dictionary keys
    model_mapping = {
        'Random Forest': RandomForestRegressor,
        'Gradient Boosting': GradientBoostingRegressor,
        'XGBoost': XGBRegressor,
        'LightGBM': LGBMRegressor,
        'CatBoost': CatBoostRegressor,
        'GPBoost': GPBoostRegressor,
        'NGBoost': NGBRegressor,
        'HistGradientBoosting': HistGradientBoostingRegressor,
        'PGBM': PGBM  # PGBM is handled separately
    }

    # Dictionary to store the best model for each type
    best_models = {}

    # Iterate over the dictionary to find the best pruner for each model type
    for (model_name, pruner), params in best_scores_autosampler.items():
        current_score = params.get('test_mse', np.inf)
        if model_name not in best_models or current_score < best_models[model_name]['score']:
            best_models[model_name] = {
                'score': current_score,
                'params': params['best_params'],
                'pruner': pruner
            }

    # Prepare a DataFrame to store predictions
    df = pd.read_csv(file_path)

    # Iterate over the best models to train and predict
    for model_name, model_info in best_models.items():
        best_params = model_info['params']
        model_class = model_mapping.get(model_name)

        if model_class is None:
            print(f"Model {model_name} is not supported or not available.")
            continue

        # Handle specific parameters or settings for model if needed
        if model_name == 'CatBoost':
            best_params.pop('verbose', None)  # Remove 'verbose' for CatBoost

        # Create an instance of the best model with the best parameters
        if model_name == 'PGBM':
            model = model_class()
            model.train((X_train, y_train), objective=mseloss_objective, metric=rmseloss_metric, params=best_params)
            predictions = model.predict(X_test)
        else:
            model = model_class(**best_params)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)

        # Add predictions to the DataFrame
        df[f'{model_name} Predictions'] = predictions

        # Plot actual vs. predicted
        plt.figure(figsize=(10, 6))
        plt.scatter(y_test, predictions, alpha=0.6)
        plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', color='red', lw=2)
        plt.xlabel("Actual")
        plt.ylabel("Predicted")
        plt.title(f"Actual vs. Predicted Values ({model_name})")
        plt.grid(True)
        plt.tight_layout()

        # Save the plot temporarily
        plot_path = f'temp_plot_{model_name}.png'
        plt.savefig(plot_path)
        plt.close()

    # Save predictions and plots to Excel
    output_file_path = "./drive/MyDrive/WATER QUALITY/EWQI_uncertainity/EWQI_ml/HyperParameter_Tuning/test_results.xlsx"
    with pd.ExcelWriter(output_file_path, engine='xlsxwriter') as writer:
        # Write data to Excel
        df.to_excel(writer, sheet_name='Data', index=False)

        # Get the xlsxwriter objects
        workbook = writer.book

        # Insert each plot into a separate worksheet
        for model_name in best_models.keys():
            # Shorten the worksheet name to fit within Excel's 31 character limit
            short_model_name = ''.join([word[0] for word in model_name.split()])  # e.g., 'RF' for 'Random Forest'
            sheet_name = f'{short_model_name}_Plot'

            worksheet = workbook.add_worksheet(sheet_name)
            writer.sheets[sheet_name] = worksheet
            plot_path = f'temp_plot_{model_name}.png'
            worksheet.insert_image('A1', plot_path)

    # Clean up temporary plot files
    for model_name in best_models.keys():
        os.remove(f'temp_plot_{model_name}.png')

    return df, best_models

# Call the function
df, best_models = get_best_models_and_predict(best_scores_autosampler, X_train, y_train, X_test, y_test, "./drive/MyDrive/WATER QUALITY/EWQI_uncertainity/test.csv")

0:	learn: 55.2702061	total: 738us	remaining: 738ms
1:	learn: 54.0060042	total: 1.38ms	remaining: 689ms
2:	learn: 52.8384528	total: 1.77ms	remaining: 588ms
3:	learn: 51.6534294	total: 2.03ms	remaining: 507ms
4:	learn: 50.7240465	total: 2.31ms	remaining: 460ms
5:	learn: 49.4556948	total: 2.6ms	remaining: 431ms
6:	learn: 48.3039953	total: 2.86ms	remaining: 405ms
7:	learn: 47.4546256	total: 3.1ms	remaining: 384ms
8:	learn: 46.4993006	total: 3.37ms	remaining: 371ms
9:	learn: 45.5309030	total: 3.62ms	remaining: 359ms
10:	learn: 44.9338270	total: 3.88ms	remaining: 349ms
11:	learn: 44.0576870	total: 4.15ms	remaining: 342ms
12:	learn: 43.3387295	total: 4.39ms	remaining: 333ms
13:	learn: 42.5921384	total: 4.64ms	remaining: 327ms
14:	learn: 41.9843798	total: 4.86ms	remaining: 319ms
15:	learn: 41.3087684	total: 5.13ms	remaining: 315ms
16:	learn: 40.7517265	total: 5.37ms	remaining: 310ms
17:	learn: 40.0547779	total: 5.63ms	remaining: 307ms
18:	learn: 39.1884171	total: 5.9ms	remaining: 305ms
19:	lea

In [21]:
plot_best_scores(best_scores_autosampler,"./drive/MyDrive/WATER QUALITY/EWQI_uncertainity/EWQI_ml/HyperParameter_Tuning/test_results.xlsx")

In [22]:
generate_interpretml_explanations_summary_pruners(best_scores_autosampler, X_train, y_train, x_test, feature_names,excel_file_path = "./drive/MyDrive/WATER QUALITY/EWQI_uncertainity/EWQI_ml/HyperParameter_Tuning/test_results.xlsx")

  0%|          | 0/47 [00:00<?, ?it/s]

  0%|          | 0/47 [00:00<?, ?it/s]

  0%|          | 0/47 [00:00<?, ?it/s]

  0%|          | 0/47 [00:00<?, ?it/s]

  0%|          | 0/47 [00:00<?, ?it/s]

  0%|          | 0/47 [00:00<?, ?it/s]

  0%|          | 0/47 [00:00<?, ?it/s]

Model HistGradientBoosting is not supported or not available.
Model PGBM is not supported or not available.
